In [1]:
import Pkg;
Pkg.activate(@__DIR__)
Pkg.status()

  Activating project at `/global/u1/b/blaschke/juliacon24-hpcworkshop/parts/mpi/explanation`


Status `/global/u1/b/blaschke/juliacon24-hpcworkshop/parts/mpi/explanation/Project.toml`
  [1520ce14] AbstractTrees v0.4.5
  [0e44f5e4] Hwloc v3.0.1
  [da04e1cc] MPI v0.20.20
  [e7922434] MPIClusterManagers v0.2.4
  [6f74fd91] NetworkInterfaceControllers v0.1.0


In [2]:
Base.active_project()

"/global/u1/b/blaschke/juliacon24-hpcworkshop/parts/mpi/explanation/Project.toml"

# Julia + MPI

([Back to Overview](../index.html#/0/8))

`MPI.jl` provides wrappers for the system MPI libraries. And the `MPIClusterManagers.jl` package lets you control MPI workflows within Julia

In [3]:
using MPI

In [4]:
MPI.versioninfo()

MPIPreferences:
  binary:  system
  abi:     MPICH
  libmpi:  libmpi_gnu_123.so
  mpiexec: srun

Package versions
  MPI.jl:             0.20.20
  MPIPreferences.jl:  0.1.11

Library information:
  libmpi:  libmpi_gnu_123.so
  libmpi dlpath:  /opt/cray/pe/lib64/libmpi_gnu_123.so
  MPI version:  3.1.0
  Library version:  
    MPI VERSION    : CRAY MPICH version 8.1.28.29 (ANL base 3.4a2)
    MPI BUILD INFO : Wed Nov 15 20:57 2023 (git hash 1cde46f)
    


In [5]:
# using Hwloc, AbstractTrees
# t = children(gettopology());

# get_nodes(tree_node, type) = filter(
#     x->x.type == type,
#     collect(AbstractTrees.PreOrderDFS(tree_node))
# )

# network_devs = filter(
#     x->Hwloc.hwloc_pci_class_string(nodevalue(x).attr.class_id) == "Ethernet",
#     get_nodes(t, :PCI_Device)
# );

In [6]:
# network_devs[1].io_children[1].object.subtype

In [7]:
using NetworkInterfaceControllers, Sockets
interfaces = NetworkInterfaceControllers.get_interface_data(IPv4)

6-element Vector{NetworkInterfaceControllers.Interface}:
 NetworkInterfaceControllers.Interface("nmn0", :v4, ip"10.100.108.84")
 NetworkInterfaceControllers.Interface("hsn0", :v4, ip"10.249.41.219")
 NetworkInterfaceControllers.Interface("hsn0:chn", :v4, ip"128.55.84.131")
 NetworkInterfaceControllers.Interface("hsn1", :v4, ip"10.249.41.203")
 NetworkInterfaceControllers.Interface("hsn2", :v4, ip"10.249.41.204")
 NetworkInterfaceControllers.Interface("hsn3", :v4, ip"10.249.41.220")

In [8]:
hsn0 = filter(x->(x.name=="hsn0:chn" && x.version==:v4), interfaces) |> only 

NetworkInterfaceControllers.Interface("hsn0:chn", :v4, ip"128.55.84.131")

In [9]:
getnameinfo(hsn0.ip)

"nid200264-hsn0"

In [10]:
# to import MPIManager
using MPIClusterManagers

# need to also import Distributed to use addprocs()
using Distributed

# specify, number of mpi workers, launch cmd, etc.
manager=MPIWorkerManager(4)

# start mpi workers and add them as julia workers too.
addprocs(
    manager,
    exeflags=`--project=$(Base.active_project())`,
    master_tcp_interface=getnameinfo(hsn0.ip)
)

4-element Vector{Int64}:
 2
 3
 4
 5

In [11]:
@mpi_do manager begin
    using MPI: MPI, Comm, Win, free
    comm = MPI.COMM_WORLD
    rank = MPI.Comm_rank(comm)
    size = MPI.Comm_size(comm)
    name = gethostname()
    println("Hello world, I am $(rank) of $(size) on $(name)")
end

      From worker 3:	Hello world, I am 1 of 4 on nid200264
      From worker 2:	Hello world, I am 0 of 4 on nid200264
      From worker 4:	Hello world, I am 2 of 4 on nid200265
      From worker 5:	Hello world, I am 3 of 4 on nid200265


In [12]:
@mpi_do manager begin
    dims = [0]
    MPI.Dims_create!(size, dims)
end

In [13]:
@mpi_do manager begin
    println(dims)
end

      From worker 4:	[4]
      From worker 5:	[4]
      From worker 3:	[4]
      From worker 2:	[4]


In [14]:
@mpi_do manager begin
    comm_cart = MPI.Cart_create(
        comm,  # MPI Communicator
        dims,  # Dimensions of grid
        [0],   # 0 == not periodic, 1 == periodic
        1,     # 0 == not allowed to reorder, 1 == allowed to reoder
    )
    me        = MPI.Comm_rank(comm_cart)
    coords    = MPI.Cart_coords(comm_cart)
    neighbors = MPI.Cart_shift(
        comm_cart,
        0,  # Which dimension to shift (zero-indexed)
        1,  # Shift magnitude
    )
end

In [15]:
@mpi_do manager begin
    println("rank=$(me); coord=$(coords), neighbors=$(neighbors)")
end

      From worker 3:	rank=1; coord=[1], neighbors=(0, 2)
      From worker 2:	rank=0; coord=[0], neighbors=(-1, 1)
      From worker 4:	rank=2; coord=[2], neighbors=(1, 3)
      From worker 5:	rank=3; coord=[3], neighbors=(2, -1)


In [16]:
MPI.PROC_NULL

-1

In [17]:
@mpi_do manager begin
    using Random
    my_int = rand(1:100)
end

In [18]:
@mpi_do manager begin
    println("rank=$(me); my_int=$(my_int)")
end

      From worker 3:	rank=1; my_int=65
      From worker 2:	rank=0; my_int=5
      From worker 5:	rank=3; my_int=76
      From worker 4:	rank=2; my_int=84


In [19]:
@mpi_do manager begin
    send_1 = zeros(Int64, 1)
    send_2 = zeros(Int64, 1)
    recv_1 = zeros(Int64, 1)
    recv_2 = zeros(Int64, 1)
end

In [20]:
@mpi_do manager begin
    # Fill send buffers (MPI uses zero-copy memory access => buffer data)
    if neighbors[1] != MPI.PROC_NULL
        copyto!(send_1, my_int)
    end
    if neighbors[2] != MPI.PROC_NULL
        copyto!(send_2, my_int)
    end 
end

In [21]:
@mpi_do manager begin
    # Placeholder for requests (so we can block on them later
    reqs = MPI.MultiRequest(4)
end

In [22]:
@mpi_do manager begin
    # Initiate receive before send
    if neighbors[1] != MPI.PROC_NULL
        MPI.Irecv!(recv_1, comm, reqs[1]; source=neighbors[1])
    end
    if neighbors[2] != MPI.PROC_NULL
        MPI.Irecv!(recv_2, comm, reqs[2]; source=neighbors[2])
    end
    # Send data
    if neighbors[1] != MPI.PROC_NULL
        MPI.Isend(send_1, comm, reqs[3]; dest=neighbors[1])
    end
    if neighbors[2] != MPI.PROC_NULL
        MPI.Isend(send_2, comm, reqs[4]; dest=neighbors[2])
    end
end

In [23]:
@mpi_do manager begin
    # Wait for all requests to finish
    MPI.Waitall(reqs)
end

In [24]:
@mpi_do manager begin
    println("rank=$(me); my_int=$(my_int); prev=$(recv_1); next=$(recv_2)")
end

      From worker 3:	rank=1; my_int=65; prev=[5]; next=[84]
      From worker 2:	rank=0; my_int=5; prev=[0]; next=[65]
      From worker 4:	rank=2; my_int=84; prev=[65]; next=[76]
      From worker 5:	rank=3; my_int=76; prev=[84]; next=[0]
